I'm pretty happy with my model's accuracy relative to GPT-4. How does it compare cost-wise?

I'll really push this to its limits -- let's see how quickly our poor model can classify the [full 2-million-recipe dataset](https://huggingface.co/datasets/corbt/all-recipes) 😈.

In [1]:
%pip install datasets==2.14.4 vllm==0.1.3


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from datasets import load_dataset

all_recipes = load_dataset("corbt/all-recipes")["train"]["input"]

print(f"Number of recipes: {len(all_recipes):,}")


Number of recipes: 2,147,248


In [5]:
from vllm import LLM, SamplingParams

llm = LLM(model="./models/run1/merged", max_num_batched_tokens=4096)

sampling_params = SamplingParams(
    # 120 should be fine for the work we're doing here.
    max_tokens=120,
    # This is a deterministic task so temperature=0 is best.
    temperature=0,
)


INFO 08-24 19:38:29 llm_engine.py:70] Initializing an LLM engine with config: model='./models/run1/merged', tokenizer='./models/run1/merged', tokenizer_mode=auto, trust_remote_code=False, dtype=torch.float16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, seed=0)
INFO 08-24 19:39:48 llm_engine.py:196] # GPU blocks: 3419, # CPU blocks: 512


In [6]:
# We'll process our recipes in batches of 10,000.

import time

BATCH_SIZE = 10000
all_outputs = []

start_time = time.time()
print(f"Start time: {start_time}")
for i in range(0, len(all_recipes), BATCH_SIZE):
    print(f"Processing recipes {i:,} to {i+BATCH_SIZE:,}...")
    outputs = llm.generate(
        all_recipes[i : i + BATCH_SIZE], sampling_params=sampling_params
    )

    all_outputs.extend([o.outputs[0].text for o in outputs])

end_time = time.time()
print(f"End time: {end_time}")
print(f"Total hours: {((end_time - start_time) / 3600):.2f}")


Start time: 1692906050.3340027
Processing recipes 0 to 10,000...


Processed prompts: 100%|██████████| 10000/10000 [04:51<00:00, 34.30it/s]


Processing recipes 10,000 to 20,000...


Processed prompts: 100%|██████████| 10000/10000 [04:54<00:00, 33.98it/s]


Processing recipes 20,000 to 30,000...


Processed prompts:  30%|███       | 3008/10000 [01:29<03:27, 33.73it/s]

Nice! I've processed all 2,147,248 recipes in under 17 hours. Let's do a cost comparison with GPT-3.5 and GPT-4. I'll use the GPT-4 latency/cost numbers based on the 5000 samples used to generate our model's training data.